# Data Visualization
# *Project 1*

### Group AA
Diana Furtado (m20200590@novaims.unl.pt); 
Hiromi Nakashima (m20201025@novaims.unl.pt); 
Miguel Martins (m20200671@novaims.unl.pt); 
Sofia Simão (m20200639@novaims.unl.pt)

#### This project aims to analyse *People's movement* dataset and understand how COVID-19 infections are impacted the mobility


*COVID Cases dataset source*: https://github.com/dssg-pt/covid19pt-data<br>
*Movement dataset source*: https://www.google.com/covid19/mobility/<br>
*Iso-codes dataset source*: https://gist.github.com/tadast/8827699

In [76]:
#Import libraries
import googlemaps
import pandas as pd
import numpy as np
import scipy
from sklearn.impute import KNNImputer
import os
from math import ceil
import glob
from pandas_profiling import ProfileReport

## Import Datasets


In [46]:
original_mobility_data = pd.read_csv("Global_Mobility_Report.csv")
original_cases_data = pd.read_csv("owid-covid-data.csv")
original_iso_map = pd.read_csv("iso_map.csv")

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Data Exploration
### Moblity Dataset

In [47]:
original_mobility_data.head()

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,place_id,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,ChIJvRKrsd9IXj4RpwoIwFYv0zM,2020-02-15,0.0,4.0,5.0,0.0,2.0,1.0
1,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,ChIJvRKrsd9IXj4RpwoIwFYv0zM,2020-02-16,1.0,4.0,4.0,1.0,2.0,1.0
2,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,ChIJvRKrsd9IXj4RpwoIwFYv0zM,2020-02-17,-1.0,1.0,5.0,1.0,2.0,1.0
3,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,ChIJvRKrsd9IXj4RpwoIwFYv0zM,2020-02-18,-2.0,1.0,5.0,0.0,2.0,1.0
4,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,ChIJvRKrsd9IXj4RpwoIwFYv0zM,2020-02-19,-2.0,0.0,4.0,-1.0,2.0,1.0


In [48]:
original_mobility_data[original_mobility_data['sub_region_1'].isna()].isna().sum()

country_region_code                                     382
country_region                                            0
sub_region_1                                          76043
sub_region_2                                          76043
metro_area                                            51348
iso_3166_2_code                                       76043
census_fips_code                                      76043
place_id                                                  0
date                                                      0
retail_and_recreation_percent_change_from_baseline     1121
grocery_and_pharmacy_percent_change_from_baseline      1150
parks_percent_change_from_baseline                     1517
transit_stations_percent_change_from_baseline          1026
workplaces_percent_change_from_baseline                 149
residential_percent_change_from_baseline               1242
dtype: int64

In [49]:
country_data = original_mobility_data[original_mobility_data['sub_region_1'].isna()]
country_data[country_data['transit_stations_percent_change_from_baseline'].isna()] #Check how some missing values are distributed

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,place_id,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
3192,AF,Afghanistan,NaN,NaN,NaN,NaN,NaN,ChIJbQL_-LZu0TgReNqWvg1GtfM,2020-08-21,NaN,NaN,12.0,NaN,1.0,1.0
3199,AF,Afghanistan,NaN,NaN,NaN,NaN,NaN,ChIJbQL_-LZu0TgReNqWvg1GtfM,2020-08-28,NaN,NaN,7.0,NaN,2.0,2.0
3206,AF,Afghanistan,NaN,NaN,NaN,NaN,NaN,ChIJbQL_-LZu0TgReNqWvg1GtfM,2020-09-04,NaN,NaN,7.0,NaN,0.0,3.0
3907,AG,Antigua and Barbuda,NaN,NaN,NaN,NaN,NaN,ChIJwdG4bbluDYwRlHmL4Yg3aic,2020-08-17,NaN,NaN,NaN,NaN,-33.0,14.0
3908,AG,Antigua and Barbuda,NaN,NaN,NaN,NaN,NaN,ChIJwdG4bbluDYwRlHmL4Yg3aic,2020-08-18,NaN,NaN,NaN,NaN,-33.0,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4464814,YE,Yemen,NaN,NaN,NaN,NaN,NaN,ChIJX7w0fKzbAxYRrneufjcp8ZI,2020-09-09,24.0,27.0,NaN,NaN,-4.0,2.0
4464815,YE,Yemen,NaN,NaN,NaN,NaN,NaN,ChIJX7w0fKzbAxYRrneufjcp8ZI,2020-09-10,28.0,27.0,NaN,NaN,3.0,4.0
4469763,ZW,Zimbabwe,NaN,NaN,NaN,NaN,NaN,ChIJYXGxBuekMRkR7r_LxV84wKE,2020-08-23,-12.0,-5.0,NaN,NaN,13.0,17.0
4469770,ZW,Zimbabwe,NaN,NaN,NaN,NaN,NaN,ChIJYXGxBuekMRkR7r_LxV84wKE,2020-08-30,-7.0,1.0,NaN,NaN,12.0,14.0


In [50]:
country_data = original_mobility_data[original_mobility_data['sub_region_1'].isna()]
country_data[country_data['country_region_code'].isna()]['country_region'].value_counts() #All missing values are from namibia

Namibia    382
Name: country_region, dtype: int64

In [51]:
original_mobility_data.dtypes

country_region_code                                    object
country_region                                         object
sub_region_1                                           object
sub_region_2                                           object
metro_area                                             object
iso_3166_2_code                                        object
census_fips_code                                      float64
place_id                                               object
date                                                   object
retail_and_recreation_percent_change_from_baseline    float64
grocery_and_pharmacy_percent_change_from_baseline     float64
parks_percent_change_from_baseline                    float64
transit_stations_percent_change_from_baseline         float64
workplaces_percent_change_from_baseline               float64
residential_percent_change_from_baseline              float64
dtype: object

### Covid cases dataset

In [52]:
original_cases_data.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
0,AFG,Asia,Afghanistan,2020-02-24,1.0,1.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
1,AFG,Asia,Afghanistan,2020-02-25,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
2,AFG,Asia,Afghanistan,2020-02-26,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
3,AFG,Asia,Afghanistan,2020-02-27,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
4,AFG,Asia,Afghanistan,2020-02-28,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511


In [53]:
original_cases_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73181 entries, 0 to 73180
Data columns (total 59 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   iso_code                               73181 non-null  object 
 1   continent                              69556 non-null  object 
 2   location                               73181 non-null  object 
 3   date                                   73181 non-null  object 
 4   total_cases                            72127 non-null  float64
 5   new_cases                              72125 non-null  float64
 6   new_cases_smoothed                     71124 non-null  float64
 7   total_deaths                           62982 non-null  float64
 8   new_deaths                             63140 non-null  float64
 9   new_deaths_smoothed                    71124 non-null  float64
 10  total_cases_per_million                71733 non-null  float64
 11  ne

In [54]:
original_cases_data.isna().sum()

iso_code                                     0
continent                                 3625
location                                     0
date                                         0
total_cases                               1054
new_cases                                 1056
new_cases_smoothed                        2057
total_deaths                             10199
new_deaths                               10041
new_deaths_smoothed                       2057
total_cases_per_million                   1448
new_cases_per_million                     1450
new_cases_smoothed_per_million            2446
total_deaths_per_million                 10580
new_deaths_per_million                   10422
new_deaths_smoothed_per_million           2446
reproduction_rate                        14249
icu_patients                             65542
icu_patients_per_million                 65542
hosp_patients                            63996
hosp_patients_per_million                63996
weekly_icu_ad

In [55]:
covid_cases_features = ['iso_code','location','date']
original_cases_data[original_cases_data['new_cases_per_million'].isna()]

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
765,ALB,Europe,Albania,2020-02-25,NaN,NaN,NaN,NaN,NaN,NaN,...,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57,0.795
766,ALB,Europe,Albania,2020-02-26,NaN,NaN,NaN,NaN,NaN,NaN,...,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57,0.795
767,ALB,Europe,Albania,2020-02-27,NaN,NaN,NaN,NaN,NaN,NaN,...,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57,0.795
768,ALB,Europe,Albania,2020-02-28,NaN,NaN,NaN,NaN,NaN,NaN,...,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57,0.795
769,ALB,Europe,Albania,2020-02-29,NaN,NaN,NaN,NaN,NaN,NaN,...,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57,0.795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67830,TCA,North America,Turks and Caicos Islands,2021-02-05,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80.22,NaN
67831,TCA,North America,Turks and Caicos Islands,2021-02-06,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80.22,NaN
67832,TCA,North America,Turks and Caicos Islands,2021-02-07,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80.22,NaN
67833,TCA,North America,Turks and Caicos Islands,2021-02-08,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80.22,NaN


In [56]:
original_cases_data.dtypes

iso_code                                  object
continent                                 object
location                                  object
date                                      object
total_cases                              float64
new_cases                                float64
new_cases_smoothed                       float64
total_deaths                             float64
new_deaths                               float64
new_deaths_smoothed                      float64
total_cases_per_million                  float64
new_cases_per_million                    float64
new_cases_smoothed_per_million           float64
total_deaths_per_million                 float64
new_deaths_per_million                   float64
new_deaths_smoothed_per_million          float64
reproduction_rate                        float64
icu_patients                             float64
icu_patients_per_million                 float64
hosp_patients                            float64
hosp_patients_per_mi

## Data preprocessing
### Iso Map (Used for tranform 2 letter iso codes into 3 letter iso codes)

In [57]:
iso_map = original_iso_map[["Alpha-2 code","Alpha-3 code"]]
iso_map = iso_map.applymap(lambda x: x.replace('"','').replace(' ',''))
iso_map.set_index("Alpha-2 code",inplace=True)
iso_map = iso_map.squeeze() #Transform into a series

### Moblity Dataset

In [62]:
mobility_features = [
    'retail_and_recreation_percent_change_from_baseline',
    'grocery_and_pharmacy_percent_change_from_baseline',
    'parks_percent_change_from_baseline',
    'transit_stations_percent_change_from_baseline',
    'workplaces_percent_change_from_baseline',
    'residential_percent_change_from_baseline',
    'date',
    'place_id',
    'country_region',
    'country_region_code'
    ]

impute_need_features = [
    'retail_and_recreation_percent_change_from_baseline',
    'grocery_and_pharmacy_percent_change_from_baseline',
    'parks_percent_change_from_baseline',
    'transit_stations_percent_change_from_baseline',
    'workplaces_percent_change_from_baseline',
    'residential_percent_change_from_baseline'
]

mobility_data = original_mobility_data[original_mobility_data['sub_region_1'].isna()] #Get only country data
mobility_data = mobility_data[mobility_data['metro_area'].isna()]
mobility_data = mobility_data[mobility_features] #Get only important data features

#Replace all nan values by NA which are from Namibia (NA is namibia country code) and Replace 2 letter iso code for 3 letter iso code
mobility_data['country_region_code'] = mobility_data['country_region_code'].fillna('NA').map(iso_map.to_dict())

mobility_data['date'] = pd.to_datetime(mobility_data['date'])
mobility_data.rename(columns={'country_region_code':'iso_code'},inplace=True)
mobility_data.reset_index(inplace=True,drop=True)

#Impute missing values with knn imputer
#All variables use the same scale so there is no need for scalling
imputer = KNNImputer(n_neighbors=5)
imputed_data = pd.DataFrame(imputer.fit_transform(mobility_data[impute_need_features]),columns = impute_need_features)
mobility_data[impute_need_features] = imputed_data
mobility_data.join
mobility_data = mobility_data.set_index(['iso_code','date'])



### Covid cases dataset

In [63]:
covid_cases_features = ['iso_code','date','new_cases_per_million','new_cases','new_deaths','new_deaths_per_million']

country_remove = ['BMU','CYM','GIB','GGY','HKG','JEY','TCA','OWID_AFR','OWID_EUR','OWID_EUN','OWID_OCE','OWID_SAM','LSO']

cases_data = original_cases_data[~original_cases_data['iso_code'].isin(country_remove)]#remove countrys with wrong data
cases_data = cases_data[covid_cases_features] #Select only usefull features
cases_data.fillna(0,inplace=True) #All missing values are 0's based on fast analysises on excel
cases_data['date'] = pd.to_datetime(cases_data['date'])
cases_data.reset_index(inplace=True,drop=True)
cases_data = cases_data.set_index(['iso_code','date'])




## Join Dataset

In [64]:
global_dataset = mobility_data.join(cases_data,how='left')
global_dataset.dropna(inplace=True)

In [65]:

#Convert place ids into longitude and latitude( Google api key is needed )
gmaps = googlemaps.Client(key='Your api key')
def getLocationArr(arrLoc):
    locs = {}
    for loc in arrLoc:
        geocode_result = gmaps.reverse_geocode(loc)
        location = geocode_result[0]['geometry']['location']
        locs[loc] = location
    return locs

locations = getLocationArr(global_dataset["place_id"].unique())


In [66]:
#Joins the latitude and longitude in a dataframe and save it
lat = []
lng = []
locs = locations.keys()
for each in locs:
    lat.append(locations[each]['lat'])
    lng.append(locations[each]['lng'])

df = pd.DataFrame({'latitude':lat,'longitude':lng},index=locs)
df.index.rename('place_id',inplace=True)
df.to_csv("locations_map.csv")



In [74]:
#export final datasets
#global_dataset = global_dataset.join(df,on='place_id').drop(columns="place_id")
path = 'extra'
if not os.path.exists(path):
    os.mkdir(path)

df.to_csv(os.path.join(path, "locations_map.csv"))
global_dataset.to_csv("Final_Dataset.csv")
cases_data.to_csv(os.path.join(path, "cases.csv"))
mobility_data.to_csv(os.path.join(path, "movement.csv"))
iso_map.to_csv(os.path.join(path, "iso_map.csv"))
global_dataset[global_dataset.isna().any(axis=1)]


,,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,country_region,new_cases_per_million,new_cases,new_deaths,new_deaths_per_million,latitude,longitude
iso_code,date,,,,,,,,,,,,,


In [77]:
ProfileReport(global_dataset).to_file("Final_Dataset.html")
ProfileReport(cases_data).to_file(os.path.join(path, "cases.html"))
ProfileReport(mobility_data).to_file(os.path.join(path, "movement.html"))

mobility_data.loc['AFG']

,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,place_id,country_region
date,,,,,,,,
2020-02-15,-9.0,-10.0,0.0,-2.0,-28.0,5.0,ChIJbQL_-LZu0TgReNqWvg1GtfM,Afghanistan
2020-02-16,3.0,11.0,1.0,10.0,4.0,0.0,ChIJbQL_-LZu0TgReNqWvg1GtfM,Afghanistan
2020-02-17,6.0,11.0,2.0,9.0,5.0,-1.0,ChIJbQL_-LZu0TgReNqWvg1GtfM,Afghanistan
2020-02-18,3.0,5.0,3.0,9.0,6.0,-1.0,ChIJbQL_-LZu0TgReNqWvg1GtfM,Afghanistan
2020-02-19,-1.0,3.0,1.0,0.0,5.0,1.0,ChIJbQL_-LZu0TgReNqWvg1GtfM,Afghanistan
...,...,...,...,...,...,...,...,...
2021-02-26,20.0,52.0,-11.0,-21.0,-19.0,0.0,ChIJbQL_-LZu0TgReNqWvg1GtfM,Afghanistan
2021-02-27,19.0,27.0,1.0,-20.0,-6.0,0.0,ChIJbQL_-LZu0TgReNqWvg1GtfM,Afghanistan
2021-02-28,41.0,85.0,5.0,-14.0,-1.0,-1.0,ChIJbQL_-LZu0TgReNqWvg1GtfM,Afghanistan
